In [1]:
import pandas as pd
import rdkit
import scipy.sparse
import os

In [2]:
## Path to file containing image features indexed with the input_compound_id
features_file = './images_files/input_hofmarcher/T_image_features_std.csv'

## Path to folder containing melloddy tuner output on image compounds
tuner_output_images = './images_files/output_files/image_model'



In [3]:
t5 = pd.read_csv(os.path.join(tuner_output_images, 'mapping_table/T5.csv'))
t5 = t5[['input_compound_id','descriptor_vector_id']]

t6_cont = pd.read_csv(os.path.join(tuner_output_images, 'results/T6_cont.csv'))
t6_cont = t6_cont[['descriptor_vector_id','cont_descriptor_vector_id']].drop_duplicates()

cmpd_mapping_table = pd.merge(t5, t6_cont, on='descriptor_vector_id', how='inner')
cmpd_mapping_table

,input_compound_id,descriptor_vector_id,cont_descriptor_vector_id
0,845253,3607,3479
1,298290,2447,2344
2,922657,4652,4464
3,62817,1122,1084
4,1001579,1481,1433
...,...,...,...
10063,773366,8753,8410
10064,989091,4554,4368
10065,26590,4650,4462
10066,736500,389,360


In [4]:
T_image_features = pd.read_csv(features_file)


In [5]:
x_features = pd.merge(T_image_features, cmpd_mapping_table, on='input_compound_id', how='right')
x_features = x_features.drop(['input_compound_id','descriptor_vector_id'], axis=1)
x_features = x_features.groupby('cont_descriptor_vector_id').agg('mean')
cls_T11_x_features = scipy.sparse.csr_matrix(x_features.values)


In [6]:
cls_T11_x = scipy.sparse.load_npz(os.path.join(tuner_output_images,'matrices/cls/cls_T11_x.npz'))


In [7]:
assert cls_T11_x_features.shape[0] == cls_T11_x.shape[0]

In [8]:
scipy.sparse.save_npz(file=os.path.join(tuner_output_images, 'matrices/cls/cls_T11_x_features.npz'), matrix=cls_T11_x_features)
